# Create Project

## Load Libraries and Functions

In [1]:
# import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


## Load Data

In [2]:
# imdb urls for datasets
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"
title_basics = 'https://datasets.imdbws.com/title.basics.tsv.gz'
title_akas = 'https://datasets.imdbws.com/title.akas.tsv.gz'
title_rating = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [3]:
# loading the data
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

# viewing the data
display(basics.head(), akas.head(), ratings.head())


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


,tconst,averageRating,numVotes
0,tt0000001,5.7,1980
1,tt0000002,5.8,265
2,tt0000003,6.5,1835
3,tt0000004,5.6,179
4,tt0000005,6.2,2624


## Cleaning the Data

In [4]:
# checking changes
display(basics.head(3), akas.head(3))

# ratings did not appear to have any NaN values
ratings.isna().sum()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0


tconst           0
averageRating    0
numVotes         0
dtype: int64

### Basics: Prepare to Guidelines and Save

#### Guidelines
* [x] Eliminate movies that are null for runtimeMinutes
* [x] Eliminate movies that are null for genre
* [x] keep only titleType==Movie
* [x] keep startYear 2000-2022
* [x] Eliminate movies that include "Documentary" in genre (see tip below)


In [5]:
#load basics
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)


In [6]:
# look at info
basics.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9930711 entries, 0 to 9930710
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 681.9+ MB


In [7]:
# replace null values
basics.replace({'\\N':np.nan}, inplace = True)

In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9930711 entries, 0 to 9930710
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 681.9+ MB


In [9]:
# 'startYear' is a string column, changing it to an float (to account for NaNs)
basics['startYear'] = basics['startYear'].astype(float)

# confirming
basics.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear         float64
endYear            object
runtimeMinutes     object
genres             object
dtype: object

In [10]:
# keep only 'Movie'
basics = basics.loc[ basics['titleType']=='movie']

In [11]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648205 entries, 8 to 9930661
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          648205 non-null  object 
 1   titleType       648205 non-null  object 
 2   primaryTitle    648205 non-null  object 
 3   originalTitle   648205 non-null  object 
 4   isAdult         648205 non-null  object 
 5   startYear       556822 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  408247 non-null  object 
 8   genres          575898 non-null  object 
dtypes: float64(1), object(8)
memory usage: 49.5+ MB


In [12]:
# filtering out nulls in genres and runtimeMinutes
basics = basics[(basics['genres'].notnull()) &
               (basics['runtimeMinutes'].notnull())
        ].copy()

In [13]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 383901 entries, 8 to 9930661
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          383901 non-null  object 
 1   titleType       383901 non-null  object 
 2   primaryTitle    383901 non-null  object 
 3   originalTitle   383901 non-null  object 
 4   isAdult         383901 non-null  object 
 5   startYear       377415 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  383901 non-null  object 
 8   genres          383901 non-null  object 
dtypes: float64(1), object(8)
memory usage: 29.3+ MB


In [14]:
# keep only startYear '2000-2022'
basics = basics [(basics['startYear']>=2000) & (basics['startYear']<=2021)]

In [15]:
# keep only documentary
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

In [16]:
# creating folder in directory
os.makedirs('Data/',exist_ok=True) 
# confirming folder creation
os.listdir("Data/")

['title_basics.csv.gz']

In [17]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

## AKA load and process
#### Guidelines
- Years from 2020-2022
- Only US movies

In [18]:
# create the pandas df
akas = pd.read_csv(title_akas, sep='\t', low_memory=True)

C:\Users\Elizabeth Spreng\AppData\Local\Temp\ipykernel_7472\1785202523.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas = pd.read_csv(title_akas, sep='\t', low_memory=True)


In [19]:
# checking info again
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36220558 entries, 0 to 36220557
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.2+ GB


In [20]:
#  checking region values for inconsistencies
akas['region'].sort_values().unique()

array(['AD', 'AE', 'AF', 'AG', 'AI', 'AL', 'AM', 'AN', 'AO', 'AQ', 'AR',
       'AS', 'AT', 'AU', 'AW', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG',
       'BH', 'BI', 'BJ', 'BM', 'BN', 'BO', 'BR', 'BS', 'BT', 'BUMM', 'BW',
       'BY', 'BZ', 'CA', 'CC', 'CD', 'CF', 'CG', 'CH', 'CI', 'CK', 'CL',
       'CM', 'CN', 'CO', 'CR', 'CSHH', 'CSXX', 'CU', 'CV', 'CW', 'CY',
       'CZ', 'DDDE', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG',
       'EH', 'ER', 'ES', 'ET', 'FI', 'FJ', 'FM', 'FO', 'FR', 'GA', 'GB',
       'GD', 'GE', 'GF', 'GH', 'GI', 'GL', 'GM', 'GN', 'GP', 'GQ', 'GR',
       'GT', 'GU', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE',
       'IL', 'IM', 'IN', 'IQ', 'IR', 'IS', 'IT', 'JE', 'JM', 'JO', 'JP',
       'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KP', 'KR', 'KW', 'KY', 'KZ',
       'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'LY',
       'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MM', 'MN', 'MO',
       'MP', 'MQ', 'MR', 'MS', 'MT', 'MU', 'MV', 

- All abbreviations appear consistent

In [21]:
# getting length of US films to compare against for confirmation
display(f"Number akas entries: {len(akas)}  \
        Number of aka US films: {len(akas[akas['region'] == 'US'])}")

'Number akas entries: 36220558          Number of aka US films: 1444985'

In [22]:
#keep only US movies
akas = akas[akas['region'] == 'US'].copy()

# confirming changes
len(akas)

1444985

In [23]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])
keepers

34803       True
42384       True
61115       True
67668       True
86800       True
           ...  
9930384     True
9930393     True
9930432    False
9930477     True
9930561    False
Name: tconst, Length: 138626, dtype: bool

In [24]:
# filter 
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
42384,tt0043139,movie,Life of a Beijing Policeman,Wo zhe yi bei zi,0,2013.0,NaN,120,"Drama,History"
61115,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67668,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86800,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
9929849,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9930244,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9930384,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9930393,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [26]:
# replace null values
akas.replace({'\\N':np.nan}, inplace = True)

## Load Ratings and process

In [27]:
# create the pandas df
rating = pd.read_csv(title_rating, sep='\t', low_memory=False)

In [28]:
# replace null values
rating.replace({'\\N':np.nan}, inplace = True)